In [1]:
using JuMP
using Gurobi

In [35]:
# type to store the info about a ContactPoint
type ContactPoint
    name::String
    rot_matrix::Array{Float64,2} # maybe should be some sort of RotationMatrix type
end

# returns the square of the L2 norm of a vector
function l2_norm(x)
    return sum(x.^2)
end

function difference_convex_functions_decomposition(l, force)
    # l = p - r, ray from com to contact location
    
    # the commas are important here to have size (2,) rather
    # than (1,2)
    a = [-l[3],l[2]]
    b = [l[3],-l[1]]
    c = [-l[2],-l[1]]
    d = force[2:3]
    e = force[[1,3]]
    f = force[1:2]

    # this is the decomposition of the cross product into difference
    # of convex functions
    # these are 3-vectors
    p = 1/4 * [l2_norm(a+d),l2_norm(b+e),l2_norm(c+f)]
    q = 1/4 * [l2_norm(a-d),l2_norm(b-e),l2_norm(c-f)]
    
    return p,q
end

difference_convex_functions_decomposition (generic function with 1 method)

In [28]:
contact_list = ["left_foot", "right_foot"]    

# initialize constants
num_timesteps = 20
num_knot_points = num_timesteps + 1
robot_mass = 1
dt = 0.1
gravity = [0,0,-9.8]
dimension = 3

#setup foot positions in world
contact_location = Dict()
contact_location["left_foot"] = [-0.5,0,0]
contact_location["right_foot"] = [0.5,0,0]
mu = 0.5


# com_data
com_data = Dict();
com_data["initial_position"] = [0,0,1]
com_data["final_position"] = [0,0,1]


# weights
weights = Dict()
weights["com_final_position"] = 100
weights["lin_momentum"] = 0.1
weights["forces"] = 0.01
weights["convex_bounds"] = 1e-4 3 # if this is too large forces will just be zero

# initialize the model
model = Model()


Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

In [29]:
# add variables to model

# com position
@variable(model, com_position[1:3, 1:num_knot_points])

# linear momentum
@variable(model, lin_momentum[1:3, 1:num_knot_points])

# angular momentum
@variable(model, ang_momentum[1:3, 1:num_knot_points])

# linear momentum dot
@variable(model, lin_momentum_dot[1:3, 1:num_knot_points])

# angular momentum dot
@variable(model, ang_momentum_dot[1:3,1:num_knot_points])

# forces
contact_list = ["left_foot", "right_foot"]
@variable(model, forces[1:3, 1:num_timesteps, contact_list] >= 0)

# will eventually need something like frames attached to those bodies so we can figure
# the friction cone and stuff
# contact_frames would be an array of rotation matrices basically that
# transform body to world


# torques
# bound them at zero for now
@variable(model, 0 <= torques[1:3, 1:num_timesteps, contact_list] <= 0)

# bounding variables for l x f terms
@variable(model, l_cross_f_plus[1:dimension, 1:num_timesteps, contact_list] >= 0)
@variable(model, l_cross_f_minus[1:dimension, 1:num_timesteps, contact_list] >= 0)

l_cross_f_minus[i,j,k] ≥ 0 ∀ i ∈ {1,2,3}, j ∈ {1,2,…,19,20}, k ∈ {left_foot,right_foot}

In [30]:
# add constraints to model
for i=1:num_timesteps
    
    # com position integration constraint
    @constraint(model, com_position[:,i+1] .- (com_position[:,i] .+
        dt/robot_mass.*lin_momentum[:,i]) .== 0)
    
    # linear momentum integration constraint
    @constraint(model, lin_momentum[:,i+1] .- (lin_momentum[:,i] 
            + lin_momentum_dot[:,i]*dt) .== 0)       
    
    # angular momentum integration constraint
    @constraint(model, ang_momentum[:, i+1] .- (ang_momentum[:,i]
            .+ ang_momentum_dot[:,i] * dt) .== 0)
    
    
    total_force = zero(AffExpr) # total force
    total_torque = zero(AffExpr) # total torque
    
    for contact_name in contact_list
        total_force += forces[:,i,contact_name]
        total_torque += torques[:,i,contact_name]
    end 
        
    # F = ma
    @constraint(model, lin_momentum_dot[:,i] - (robot_mass*gravity .+ total_force) .== 0)
    
    # the total l x f term
    l_cross_f = zero(AffExpr)
    for contact_name in contact_list
        # vector from com --> contact location
        l = contact_location[contact_name] .- com_position[:,i]
        force_local = forces[:,i,contact_name]
        
        # decompose into difference of convex functions
        p,q = difference_convex_functions_decomposition(l, force_local)
        
        # add constraint for convex relaxation
        @constraint(model, p .<= l_cross_f_plus[:,i,contact_name])
        @constraint(model, q .<= l_cross_f_minus[:,i,contact_name])
        
        # add the contribution of this contact to the overally l x f term.
        l_cross_f += l_cross_f_plus[:,i,contact_name] - l_cross_f_minus[:,i,contact_name]
    end
    
    # angular momentum dot constraint
    @constraint(model, ang_momentum_dot[:,i] .- (l_cross_f .+ total_torque) .== 0)
end

# initial condition constraints
@constraint(model, com_position[:,1] .== com_data["initial_position"])
@constraint(model, lin_momentum[:,1] .== zeros(3))
@constraint(model, lin_momentum_dot[:,1] .== zeros(3))

3-element Array{JuMP.ConstraintRef{JuMP.Model,JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}},1}:
 lin_momentum_dot[1,1] = 0
 lin_momentum_dot[2,1] = 0
 lin_momentum_dot[3,1] = 0

In [31]:
# add objective
com_final_position_cost = weights["com_final_position"]*
l2_norm(com_position[:,end] - com_data["final_position"])

lin_momentum_cost = weights["lin_momentum"]*l2_norm(lin_momentum[:,:,:])    

force_cost = zero(QuadExpr)
convex_bound_cost = zero(QuadExpr)
for contact_name in contact_list
    force_cost += weights["forces"]*l2_norm(forces[:,:,contact_name])
    convex_bound_cost += weights["convex_bounds"]*l2_norm(l_cross_f_plus[:,:,contact_name])
    convex_bound_cost += weights["convex_bounds"]*l2_norm(l_cross_f_minus[:,:,contact_name])
end

total_objective = com_final_position_cost + lin_momentum_cost
+ force_cost

total_objective += convex_bound_cost

@objective(model, Min, total_objective)

:Min

In [32]:
# solve the model
status = solve(model)

Optimize a model with 309 rows, 795 columns and 1149 nonzeros
Model has 306 quadratic objective terms
Model has 240 quadratic constraints
Presolve removed 197 rows and 328 columns
Presolve time: 0.02s
Presolved: 1353 rows, 1391 columns, 3266 nonzeros
Presolved model has 237 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 Free vars  : 51
 AA' NZ     : 8.431e+03
 Factor NZ  : 2.629e+04 (roughly 1 MByte of memory)
 Factor Ops : 6.144e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.12472556e+01  5.12472556e+01  6.42e+01 3.77e+02  4.02e+01     0s
   1  -2.17363677e+03 -1.61205117e+02  5.08e+01 1.33e+02  2.84e+01     0s
   2  -2.18205130e+03 -2.30738687e+02  2.87e+01 8.92e+01  1.63e+01     0s
   3  -2.17111874e+03 -2.35900316e+02  2.69e+01 8.61e+01  1.53e+01     0s
   4  -1.81344816e+03 -2.74765859e+02  2.12e+0

:Optimal

In [39]:
# parse solution data
# trailing underscore indicates value of those elements
com_position_ = getvalue(com_position)
lin_momentum_ = getvalue(lin_momentum)
ang_momentum_ = getvalue(ang_momentum)
lin_momentum_dot_ = getvalue(lin_momentum_dot)
ang_momentum_dot_ = getvalue(ang_momentum_dot)
l_cross_f_plus_ = getvalue(l_cross_f_plus)
l_cross_f_minus_ = getvalue(l_cross_f_minus)


# should be same size as lin_momentum_dot_
p_ = zeros(ang_momentum_dot_)
q_ = zeros(ang_momentum_dot_)

# populate q_, v_ check tightness with l_cross_f_plus

3×21 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [9]:
getvalue(lin_momentum)

3×21 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [34]:
x

3×21 Array{Float64,2}:
 -0.0  -0.0  -0.0  2.01796e-7  6.31566e-7  …  0.0001499    0.00020549
 -0.0  -0.0  -0.0  2.01075e-7  6.29107e-7     0.000151902  0.00020929
  1.0   1.0   1.0  0.999854    0.999775       1.01194      0.999636  